In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS

import tqdm as tqdm

from models_repo import load_checkpoint, load_model

from data_sets import get_dataset
from helper_functions import show_random_images

from torch.optim import Adam

c:\Users\Steffen\.conda\envs\pytorch_1\Lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (4.1.1) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Using device: ", device)

Using device:  cuda


In [3]:
from models_repo import CNN_MNIST

In [4]:
def bayesian_model(x_data, y_data, model):
    # Define priors for each parameter in the network
    priors = {
        'conv1.weight': dist.Normal(torch.zeros_like(model.conv1.weight), torch.ones_like(model.conv1.weight)).to_event(4),
        'conv1.bias': dist.Normal(torch.zeros_like(model.conv1.bias), torch.ones_like(model.conv1.bias)).to_event(1),
        'conv2.weight': dist.Normal(torch.zeros_like(model.conv2.weight), torch.ones_like(model.conv2.weight)).to_event(4),
        'conv2.bias': dist.Normal(torch.zeros_like(model.conv2.bias), torch.ones_like(model.conv2.bias)).to_event(1),
        'fc1.weight': dist.Normal(torch.zeros_like(model.fc1.weight), torch.ones_like(model.fc1.weight)).to_event(2),
        'fc1.bias': dist.Normal(torch.zeros_like(model.fc1.bias), torch.ones_like(model.fc1.bias)).to_event(1),
        'fc2.weight': dist.Normal(torch.zeros_like(model.fc2.weight), torch.ones_like(model.fc2.weight)).to_event(2),
        'fc2.bias': dist.Normal(torch.zeros_like(model.fc2.bias), torch.ones_like(model.fc2.bias)).to_event(1),
    }
    lifted_module = pyro.random_module("module", model, priors)
    lifted_bnn = lifted_module()

    with pyro.plate("data", len(x_data)):
        prediction_logits = lifted_bnn(x_data)
        pyro.sample("obs", dist.Categorical(logits=prediction_logits), obs=y_data)


def bayesian_model_small(x_data, y_data, model):
    priors = {
        'fc1.weight': dist.Normal(torch.zeros_like(model.fc1.weight), torch.ones_like(model.fc1.weight)).to_event(2),
        'fc1.bias': dist.Normal(torch.zeros_like(model.fc1.bias), torch.ones_like(model.fc1.bias)).to_event(1),
        'fc2.weight': dist.Normal(torch.zeros_like(model.fc2.weight), torch.ones_like(model.fc2.weight)).to_event(2),
        'fc2.bias': dist.Normal(torch.zeros_like(model.fc2.bias), torch.ones_like(model.fc2.bias)).to_event(1),
    }

    lifted_module = pyro.random_module("module", model, priors)
    lifted_bnn = lifted_module()

    with pyro.plate("data", len(x_data)):
        prediction_logits = lifted_bnn(x_data)
        pyro.sample("obs", dist.Categorical(logits=prediction_logits), obs=y_data)

def run_mcmc(x_data, y_data, model, num_samples=500, warmup_steps=200):
    nuts_kernel = NUTS(lambda x, y: bayesian_model(x, y, model))
    mcmc = MCMC(nuts_kernel, num_samples=num_samples, warmup_steps=warmup_steps)
    mcmc.run(x_data, y_data)
    return mcmc


def run_mcmc_small(x_data, y_data, model, num_samples=500, warmup_steps=200):
    nuts_kernel = NUTS(lambda x, y: bayesian_model_small(x, y, model))
    mcmc = MCMC(nuts_kernel, num_samples=num_samples, warmup_steps=warmup_steps)
    mcmc.run(x_data, y_data)
    return mcmc

def run_mcmc_all(data_loader, model, num_samples=500, warmup_steps=200):
    all_samples = []
    for x_data, y_data in data_loader:
        x_data, y_data = x_data.to(device), y_data.to(device)
        nuts_kernel = NUTS(lambda x, y: bayesian_model(x, y, model))
        mcmc = MCMC(nuts_kernel, num_samples=num_samples, warmup_steps=warmup_steps)
        mcmc.run(x_data, y_data)
        all_samples.append(mcmc.get_samples())
    return all_samples

def run_mcmc_small_all(data_loader, model, num_samples=500, warmup_steps=200):
    all_samples = []
    for x_data, y_data in data_loader:
        x_data, y_data = x_data.to(device), y_data.to(device)
        nuts_kernel = NUTS(lambda x, y: bayesian_model_small(x, y, model))
        mcmc = MCMC(nuts_kernel, num_samples=num_samples, warmup_steps=warmup_steps)
        mcmc.run(x_data, y_data)
        all_samples.append(mcmc.get_samples())
    return all_samples

In [5]:
import GPUtil
def get_gpu_usage():
    gpus = GPUtil.getGPUs()
    gpu = gpus[0]
    return {
        "id": gpu.id,
        "name": gpu.name,
        "load": gpu.load * 100,
        "memory_free": gpu.memoryFree / 1024,
        "memory_used": gpu.memoryUsed / 1024,
        "memory_total": gpu.memoryTotal / 1024,
        "temperature": gpu.temperature
    }

def print_gpu_usage(gpu_info):
    print(f"GPU ID: {gpu_info['id']}")
    print(f"GPU Name: {gpu_info['name']}")
    print(f"GPU Load: {gpu_info['load']:.2f}%")
    print(f"GPU Free Memory: {gpu_info['memory_free']:.2f} GB")
    print(f"GPU Used Memory: {gpu_info['memory_used']:.2f} GB")
    print(f"GPU Total Memory: {gpu_info['memory_total']:.2f} GB")
    print(f"GPU Temperature: {gpu_info['temperature']} °C")

def run_mcmc_all_GPU(data_loader, model, num_samples=500, warmup_steps=200, batch_limit=2):
    all_samples = []
    gpu_usage_info = []
    for i, (x_data, y_data) in enumerate(data_loader):
        if i >= batch_limit:
            break
        x_data, y_data = x_data.to(device), y_data.to(device)
        
        before_gpu_info = get_gpu_usage()
        print("Before running MCMC:")
        print_gpu_usage(before_gpu_info)
        
        nuts_kernel = NUTS(lambda x, y: bayesian_model(x, y, model))
        mcmc = MCMC(nuts_kernel, num_samples=num_samples, warmup_steps=warmup_steps)
        mcmc.run(x_data, y_data)
        all_samples.append(mcmc.get_samples())
        
        after_gpu_info = get_gpu_usage()
        print("After running MCMC:")
        print_gpu_usage(after_gpu_info)
        
        gpu_usage_info.append({
            "before": before_gpu_info,
            "after": after_gpu_info
        })
        
    return all_samples, gpu_usage_info

def run_mcmc_small_all_GPU(data_loader, model, num_samples=500, warmup_steps=200, batch_limit=2):
    all_samples = []
    gpu_usage_info = []
    for i, (x_data, y_data) in enumerate(data_loader):
        if i >= batch_limit:
            break
        x_data, y_data = x_data.to(device), y_data.to(device)
        
        before_gpu_info = get_gpu_usage()
        print("Before running MCMC:")
        print_gpu_usage(before_gpu_info)
        
        nuts_kernel = NUTS(lambda x, y: bayesian_model_small(x, y, model))
        mcmc = MCMC(nuts_kernel, num_samples=num_samples, warmup_steps=warmup_steps)
        mcmc.run(x_data, y_data)
        all_samples.append(mcmc.get_samples())
        
        after_gpu_info = get_gpu_usage()
        print("After running MCMC:")
        print_gpu_usage(after_gpu_info)
        
        gpu_usage_info.append({
            "before": before_gpu_info,
            "after": after_gpu_info
        })
        
    return all_samples, gpu_usage_info

In [6]:
MNIST_train, MNIST_test, MNIST_val, MNIST_paramters = get_dataset('MNIST','data',batch_size=128, validation_split=0.2)

Data set MNIST found
Loading data set...
Directory data\MNIST already exists


In [7]:
opti_type = Adam
path_CNN_MNIST = r"Done_test\test_1_comparison\checkpoints\CNN_mnist_epoch_40.pth"
CNN_MODEL_MNIST, CNN_OPTI_MNIST = load_checkpoint(CNN_MNIST, opti_type, path_CNN_MNIST)
CNN_MODEL_MNIST = CNN_MODEL_MNIST.to(device)

Checkpoint loaded from Done_test\test_1_comparison\checkpoints\CNN_mnist_epoch_40.pth


In [8]:
x_data, y_data = next(iter(MNIST_test))
x_data, y_data = x_data.to(device), y_data.to(device)
mcmc_MINIST = run_mcmc(x_data, y_data, CNN_MODEL_MNIST)

Warmup:   0%|          | 0/700 [00:00, ?it/s]c:\Users\Steffen\.conda\envs\pytorch_1\Lib\site-packages\pyro\primitives.py:526: FutureWarning: The `random_module` primitive is deprecated, and will be removed in a future release. Use `pyro.nn.Module` to create Bayesian modules from `torch.nn.Module` instances.
  warnings.warn(
Sample: 100%|██████████| 700/700 [1:55:02,  9.86s/it, step size=1.78e-07, acc. prob=0.776]


In [9]:
#x_data, y_data = next(iter(MNIST_test))
#x_data, y_data = x_data.to(device), y_data.to(device)
#mcmc_MINIST = run_mcmc(x_data, y_data, CNN_MODEL_MNIST)
mcmc_MINIST, GPU_ALL = run_mcmc_all_GPU(MNIST_train, CNN_MODEL_MNIST, batch_limit=5)

Before running MCMC:
GPU ID: 0
GPU Name: NVIDIA GeForce RTX 3060
GPU Load: 40.00%
GPU Free Memory: 6.51 GB
GPU Used Memory: 5.32 GB
GPU Total Memory: 12.00 GB
GPU Temperature: 61.0 °C


Sample: 100%|██████████| 700/700 [1:54:39,  9.83s/it, step size=7.27e-08, acc. prob=0.843]


After running MCMC:
GPU ID: 0
GPU Name: NVIDIA GeForce RTX 3060
GPU Load: 39.00%
GPU Free Memory: 5.74 GB
GPU Used Memory: 6.09 GB
GPU Total Memory: 12.00 GB
GPU Temperature: 53.0 °C
Before running MCMC:
GPU ID: 0
GPU Name: NVIDIA GeForce RTX 3060
GPU Load: 33.00%
GPU Free Memory: 5.74 GB
GPU Used Memory: 6.10 GB
GPU Total Memory: 12.00 GB
GPU Temperature: 53.0 °C


Sample: 100%|██████████| 700/700 [1:54:23,  9.80s/it, step size=1.73e-07, acc. prob=0.868]


After running MCMC:
GPU ID: 0
GPU Name: NVIDIA GeForce RTX 3060
GPU Load: 39.00%
GPU Free Memory: 4.95 GB
GPU Used Memory: 6.88 GB
GPU Total Memory: 12.00 GB
GPU Temperature: 45.0 °C
Before running MCMC:
GPU ID: 0
GPU Name: NVIDIA GeForce RTX 3060
GPU Load: 41.00%
GPU Free Memory: 4.95 GB
GPU Used Memory: 6.89 GB
GPU Total Memory: 12.00 GB
GPU Temperature: 45.0 °C


...

After running MCMC:
GPU ID: 0
GPU Name: NVIDIA GeForce RTX 3060
GPU Load: 39.00%
GPU Free Memory: 4.44 GB
GPU Used Memory: 7.39 GB
GPU Total Memory: 12.00 GB
GPU Temperature: 61.0 °C
Before running MCMC:
GPU ID: 0
GPU Name: NVIDIA GeForce RTX 3060
GPU Load: 39.00%
GPU Free Memory: 4.44 GB
GPU Used Memory: 7.39 GB
GPU Total Memory: 12.00 GB
GPU Temperature: 61.0 °C


Sample: 100%|██████████| 700/700 [2:11:30, 11.27s/it, step size=5.96e-08, acc. prob=0.827]


After running MCMC:
GPU ID: 0
GPU Name: NVIDIA GeForce RTX 3060
GPU Load: 36.00%
GPU Free Memory: 3.31 GB
GPU Used Memory: 8.52 GB
GPU Total Memory: 12.00 GB
GPU Temperature: 54.0 °C
Before running MCMC:
GPU ID: 0
GPU Name: NVIDIA GeForce RTX 3060
GPU Load: 40.00%
GPU Free Memory: 3.31 GB
GPU Used Memory: 8.52 GB
GPU Total Memory: 12.00 GB
GPU Temperature: 54.0 °C


Sample: 100%|██████████| 700/700 [2:06:29, 10.84s/it, step size=1.11e-07, acc. prob=0.770]

After running MCMC:
GPU ID: 0
GPU Name: NVIDIA GeForce RTX 3060
GPU Load: 40.00%
GPU Free Memory: 2.45 GB
GPU Used Memory: 9.39 GB
GPU Total Memory: 12.00 GB
GPU Temperature: 52.0 °C
